__Non Linear Outputs__

After fitting a large suite of data with the same pipeline, the aggregator allows us to load the results and manipulate / plot them using a Python script or Jupyter notebook.

This script uses the results generated by the script '/autolens_workspace/aggregator/beginner_runner.py', which fitted 3 simulated strong lenses. We fitted each image with the pipeline:

'autolens_workspace/pipelines/beginner/no_lens_light/lens_sie__source_inversion.py'

This pipeline is composed of 3 phases.

__File Output__

The results of this fit are in the '/output/aggregator_sample_beginner' folder. First, take a look in this folder. Provided you haven't rerun the runner, you'll notice that all the results (e.g. optimizer, optimizer_backup, model.results, images, etc.) are in .zip files as opposed to folders that can be instantly accessed.

This is because when the pipeline was run, the 'remove_files' option in the 'config/general.ini' was set to True. This means all results (other than the .zip file) were removed. This feature is implemented because super-computer often have a limit on the number of files allowed per user.

Bare in mind the fact that all results are in .zip files - we'll come back to this point in a second.

__Aggregator__

We can load the results of each pipeline's analysis of each of the 3 images using the aggregator. This will allow us to manipulate the results in this Python script (or a Jupyter notebook) to plot figures, interpret results, check specific values, etc.

In [ ]:
import autofit as af

To begin, we setup the path to the output path we want to load results from, which in this case is the folder 'autolens_workspace/output/aggregator_sample_beginner'.

In [ ]:
workspace_path = "/home/jammy/PycharmProjects/PyAuto/autolens_workspace/"
output_path = workspace_path + "output"
aggregator_results_path = output_path + "/aggregator_sample_beginner"

Now we'll use this path to explicitly set the config path and output path.

In [ ]:
af.conf.instance = af.conf.Config(
    config_path=str(workspace_path + "/config"), output_path=str(output_path)
)

To set up the aggregator we simply pass it the folder of the results we want to load.

In [ ]:
aggregator = af.Aggregator(directory=str(aggregator_results_path))

Before we continue, take another look at the output folder. The .zip files containing results have now all been unzipped, such that the results are accessible on your laptop for navigation. This means you can run fits to many lenses on a super computer and easily unzip all the results on your computer afterwards via the aggregator.

We can now create a list of the 'non-linear outputs' of every fit. An instance of the NonLinearOutput class acts as an interface between the results of the non-linear fit on your hard-disk and Python.

The fits to each lens used MultiNest, so below we create a list of instances of the MultiNestOutput class (the non-linear output class will change dependent on the non-linear optimizer used).

In [ ]:
outputs = aggregator.output

When we print this list of outputs we see 9 different MultiNestOutput instances. These correspond to all 3 phases of each pipeline's fit to all 3 images.

In [ ]:
print("MultiNest Outputs: \n")
print(outputs)
print()
print("Total Outputs = ", len(outputs), "\n")

If we want to remove the fits of the first 2 phases and just keep the MultiNestOutputs of the 3rd and final phase of the pipeline we can do so by filtering for the phase's name.

In [ ]:
phase_name = "phase_3__source_inversion"
outputs = aggregator.filter(phase=phase_name).output

As expected, this list now has only 3 MultiNestOutputs, one for each image we fitted.

In [ ]:
print("Phase Name Filtered MultiNest Outputs: \n")
print(outputs)
print()
print("Total Outputs = ", len(outputs), "\n")

In this example, we only fitted the 3 images using one pipeline. But suppose we used multiple pipelines, like we do in the advanced pipelines. In this case, the aggregator would load the MultiNestOutputs of all fits of all phases of all pipelines!

In such circumstances, we can filter by pipeline name.

In [ ]:
pipeline_name = "pipeline__lens_sie__source_inversion"
outputs = aggregator.filter(pipeline=pipeline_name, phase=phase_name).output

As expected, this list again has 3 MultiNestOutputs.

In [ ]:
print("Pipeline Name Filtered MultiNest Outputs: \n")
print(outputs)
print()
print("Total Outputs = ", len(outputs), "\n")

We can use the outputs to create a list of the most-likely (e.g. highest likelihood) model of each fit to our three images (in this case in phase 3).

In [ ]:
most_likely_vector = [
    out.most_probable_vector for out in outputs
]
print("Most Likely Model Parameter Lists: \n")
print(most_likely_vector, "\n")

This provides us with lists of all model parameters. However, this isn't that much use - which values correspond to which parameters?

Its more useful to create the model instance of every fit.

In [ ]:
instances = [out.most_likely_instance for out in outputs]
print("Most Likely Model Instances: \n")
print(instances, "\n")

A model instance contains all the model components of our fit - most importantly the list of galaxies we specified in the pipeline.

In [ ]:
print(instances[0].galaxies)
print(instances[1].galaxies)
print(instances[2].galaxies)

These galaxies will be named according to the pipeline (in this case, 'lens' and 'source').

In [ ]:
print(instances[0].galaxies.lens)
print()
print(instances[1].galaxies.source)

Their light and mass profiles are also named according to the pipeline.

In [ ]:
print(instances[0].galaxies.lens.mass)
print()
print(instances[0].galaxies.lens.shear)
print()
print(instances[1].galaxies.source.pixelization)
print()
print(instances[1].galaxies.source.regularization)

We can also access the 'most probable' model, which is the model computed by marginalizing over the MultiNest samples of every parameter in 1D and taking the median of this PDF.

In [ ]:
mp_vector = [out.most_probable_vector for out in outputs]
mp_instances = [out.most_probable_instance for out in outputs]

print("Most Probable Model Parameter Lists: \n")
print(mp_vector, "\n")
print("Most probable Model Instances: \n")
print(mp_instances, "\n")
print(mp_instances[0].galaxies.lens.mass)
print(mp_instances[0].galaxies.lens.shear)
print()

We can compute the model parameters at a given sigma value (e.g. at 3.0 sigma limits).

These parameter values do not account for covariance between the model. For example if two parameters are degenerate this will find their values from the degeneracy in the 'same direction' (e.g. both will be positive). 

In [ ]:
upper_vectors = [out.vector_at_upper_sigma(sigma=3.0) for out in outputs]

upper_instances = [out.instance_at_upper_sigma(sigma=3.0) for out in outputs]
                                               
lower_vectors = [out.vector_at_lower_sigma(sigma=3.0) for out in outputs]
                                               
lower_instances = [out.instance_at_lower_sigma(sigma=3.0) for out in outputs]

print("Errors Lists: \n")
print(upper_vectors, "\n")
print(lower_vectors, "\n")
print("Errors Instances: \n")
print(upper_instances, "\n")
print(lower_instances, "\n")

We can compute the upper and lower errors on each parameter at a given sigma limit.

In [ ]:
# These do not currently work due to a bug which will be fixed on Friday

upper_errors = [out.error_vector_at_upper_sigma(sigma=3.0) for out in outputs]
upper_error_instances = [
    out.error_instance_at_upper_sigma(sigma=3.0) for out in outputs
]
lower_errors = [out.error_vector_at_lower_sigma(sigma=3.0) for out in outputs]
lower_error_instances = [
    out.error_instance_at_lower_sigma(sigma=3.0) for out in outputs
]

print("Errors Lists: \n")
print(upper_errors, "\n")
print(lower_errors, "\n")
print("Errors Instances: \n")
print(upper_error_instances, "\n")
print(lower_error_instances, "\n")

The maximum likelihood of each model fit and its Bayesian evidence (estimated via MultiNest) are also available.

Given each fit is to a different image, these are not very useful. However, in tutorial 5 we'll look at using the aggregator for images that we fit with many different models and many different pipelines, in which case comparing the evidences allows us to perform Bayesian model comparison!

In [ ]:
print("Likelihoods: \n")
print([out.maximum_log_likelihood for out in outputs])
print([out.evidence for out in outputs])

We can also print the "model_results" of all phases, which is string that summarizes every fit's lens model providing quick inspection of all results.

In [ ]:
results = aggregator.filter(pipeline=pipeline_name).model_results
print("Model Results Summary: \n")
print(results, "\n")